<a href="https://colab.research.google.com/github/rtindru/petrichor/blob/master/Master_FastText_Wiki_Pretrained_Embed_Petrichor_LSTM_30.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
# This is location of the data
data_url = '/content/gdrive/My Drive/petrichor_new/master_dataset.csv'
vector_path = '/content/gdrive/My Drive/petrichor_new/wiki-news-300d-1M.vec'

In [ ]:
# Top Level Imports

import pandas as pd
import numpy as np
import nltk

from nltk.corpus import stopwords
from nltk.tokenize import wordpunct_tokenize
from keras.preprocessing.text import text_to_word_sequence

In [ ]:
# Import the data

# We have two different dictonaries, let's just merge them together
df_one = pd.read_csv(data_url, sep=',', error_bad_lines=False, encoding='latin1', header=None, names=['word', 'pos', 'meaning'])
df_two = pd.read_csv('https://raw.githubusercontent.com/rtindru/petrichor/master/dataset.csv')
df = pd.concat([df_one, df_two]).drop('idx', axis=1)

In [ ]:
print(df.info())
print('---')
print(df.word.value_counts()[:10])

In [ ]:
df['label'] = df.word.str.lower()
print(len(df.label.unique()))

262201


In [ ]:
df['count'] = df[['label', 'meaning']].groupby(['label']).transform('count')
downsized_df = df[(df['count'] > 30)]
print('Predicting on {} samples of data'.format(downsized_df.shape))
print('Num of unique classes: {}'.format(len(downsized_df.label.unique())))

Predicting on (226070, 5) samples of data
Num of unique classes: 4233


In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
MAX_NB_WORDS = 50000

tokenizer = Tokenizer(num_words=-1)
tokenizer.fit_on_texts(downsized_df.meaning)
sequences = tokenizer.texts_to_sequences(downsized_df.meaning)
word_index = tokenizer.word_index

print('Found %s unique tokens.' % len(word_index))


# Assigned maxlen based on the third quartile of frequency distribution of the meaning lengths
downsized_df['wordlen'] = downsized_df.meaning.apply(len)
print(downsized_df.wordlen.describe())
MAX_SEQUENCE_LENGTH = 80

X = pd.DataFrame(pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH))

Found 35195 unique tokens.
count    226070.000000
mean         63.732052
std          49.893732
min           3.000000
25%          31.000000
50%          50.000000
75%          82.000000
max         942.000000
Name: wordlen, dtype: float64


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
# import keras.backend as K
from sklearn.preprocessing import LabelEncoder


le = LabelEncoder()
y = pd.DataFrame(le.fit_transform(downsized_df.label))
number_of_classes = len(le.classes_)

"""
# Use train_test_split from scikit
# Checkout stratified sampling & splitting
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state = 42)

labels = K.one_hot(K.cast(y,'uint8'), number_of_classes)
y_train = K.one_hot(K.cast(y_train,'uint8'), number_of_classes)
y_test = K.one_hot(K.cast(y_test,'uint8'), number_of_classes)

print('Shape of data tensor:', X.shape)
print('Shape of label tensor:', y.shape)
"""

"\n# Use train_test_split from scikit\n# Checkout stratified sampling & splitting\nfrom sklearn.model_selection import train_test_split\nX_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state = 42)\n\nlabels = K.one_hot(K.cast(y,'uint8'), number_of_classes)\ny_train = K.one_hot(K.cast(y_train,'uint8'), number_of_classes)\ny_test = K.one_hot(K.cast(y_test,'uint8'), number_of_classes)\n\nprint('Shape of data tensor:', X.shape)\nprint('Shape of label tensor:', y.shape)\n"

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

_X_train, X_test, _y_train, y_test = train_test_split(X, y, test_size=0.33, random_state = 42)

X_train, X_valid, y_train, y_valid = train_test_split(_X_train, _y_train, test_size=0.25, random_state = 42)


print('Shape of data tensor:', X.shape)
print('Shape of label tensor:', y.shape)

Shape of data tensor: (226070, 80)
Shape of label tensor: (226070, 1)


In [ ]:
import numpy as np
import keras
from tensorflow.keras.utils import Sequence

class DataGenerator(Sequence):
    'Generates data for Keras'
    def __init__(self, X, labels, batch_size=32, dim=(32,32,32), n_channels=1,
                 n_classes=10, shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.X = X
        self.list_IDs = X.index.values
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]
        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, *self.dim))
        y = np.empty((self.batch_size), dtype=int)

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
            X[i,] = self.X.loc[ID]

            # Store class
            y[i] = self.labels.loc[ID]

        return X, keras.utils.to_categorical(y, num_classes=self.n_classes)


In [ ]:
params = {'dim': (80,),
          'batch_size': 32,
          'n_classes': number_of_classes,
          'n_channels': 1,
          'shuffle': True}


# Generators
training_generator = DataGenerator(X_train, y_train, **params)
validation_generator = DataGenerator(X_valid, y_valid, **params)

In [ ]:
import gensim
gn_model = gensim.models.KeyedVectors.load_word2vec_format(vector_path)
# gn_model = gensim.models.KeyedVectors.load_word2vec_format('/content/gdrive/My Drive/petrichor_new/GoogleNews-vectors-negative300.bin.gz', binary=True)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [ ]:
def get_coefficients(word, model):
  """
  Helper method to return coeffs for a model; or zeros!
  """
  try:
    return model.get_vector(word)
  except KeyError:
    return np.zeros(model.wv.vector_size)

In [ ]:
# Preload the embedding matrix
EMBEDDING_DIM = 300  # same as the lenght of the keyed vector

embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_matrix[i] = get_coefficients(word, gn_model)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


In [ ]:
from tensorflow.python.keras.layers import Dense, Embedding, Dropout, SpatialDropout1D, Bidirectional, LSTM
from tensorflow.python.keras import Sequential

TRAIN_SAMPLES = len(X_train)
LSTM_DIM = 128 
NUM_CLASSES = number_of_classes

model = Sequential()
model.add(Embedding(input_dim=len(word_index)+1, output_dim=EMBEDDING_DIM, weights=[embedding_matrix], input_length=MAX_SEQUENCE_LENGTH))
model.add(LSTM(LSTM_DIM, dropout=0.2, recurrent_dropout=0.2))
# model.add(LSTM(LSTM_DIM))
model.add(Dense(NUM_CLASSES, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy', 'top_k_categorical_accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 80, 300)           10558800  
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               219648    
_________________________________________________________________
dense_1 (Dense)              (None, 4233)              546057    
Total params: 11,324,505
Trainable params: 11,324,505
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
model.fit(x=training_generator, validation_data=validation_generator, batch_size=64, verbose=True, epochs=100, use_multiprocessing=True)

Epoch 1/100
3549/3549 [==============================] - 1139s 321ms/step - loss: 8.2844 - accuracy: 0.0014 - top_k_categorical_accuracy: 0.0069 - val_loss: 8.2603 - val_accuracy: 0.0018 - val_top_k_categorical_accuracy: 0.0076
Epoch 2/100
3549/3549 [==============================] - 1100s 310ms/step - loss: 8.2402 - accuracy: 0.0018 - top_k_categorical_accuracy: 0.0075 - val_loss: 8.2592 - val_accuracy: 0.0017 - val_top_k_categorical_accuracy: 0.0075
Epoch 3/100
3549/3549 [==============================] - 1099s 310ms/step - loss: 8.2364 - accuracy: 0.0016 - top_k_categorical_accuracy: 0.0075 - val_loss: 8.2582 - val_accuracy: 0.0018 - val_top_k_categorical_accuracy: 0.0079
Epoch 4/100
3549/3549 [==============================] - 1097s 309ms/step - loss: 8.2342 - accuracy: 0.0018 - top_k_categorical_accuracy: 0.0076 - val_loss: 8.2583 - val_accuracy: 0.0018 - val_top_k_categorical_accuracy: 0.0075
Epoch 5/100
3549/3549 [==============================] - 1088s 306ms/step - loss: 8.2328

In [ ]:
# from IPython.display import SVG
# from keras.utils.vis_utils import model_to_dot

# SVG(model_to_dot(model, show_shapes=True, show_layer_names=False, 
#                  rankdir='LR').create(prog='dot', format='svg'))

In [ ]:
# """
# steps_per_epoch = TotalTrainingSamples / TrainingBatchSize
# validation_steps = TotalvalidationSamples / ValidationBatchSize
# """
# EPOCHS = 5
# BATCH_SIZE = 64
# STEPS_PER_EPOCH = TRAIN_SAMPLES // BATCH_SIZE
# VAL_SPLIT = 0.2 
# VALIDATION_STEPS = VAL_SPLIT * TRAIN_SAMPLES // BATCH_SIZE

# # https://blog.keras.io/using-pre-trained-word-embeddings-in-a-keras-model.html

# model.fit(X_train, y_train, epochs=EPOCHS,
#           shuffle=True, verbose=True)

# print(model.metrics_names)
# print(model.evaluate(X_test, y_test))

In [ ]:
model.evaluate(X_test, keras.utils.to_categorical(y_test, num_classes=number_of_classes))